In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import time
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [2]:
path_to_file = 'bad_advice.txt'

In [3]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

print('Length of text: {} characters'.format(len(text)))

Length of text: 44248 characters


In [4]:
text = text.split('\r\n\r\n\r\n')

In [5]:
text[5]

'Потерявшийся ребенок\r\n\r\nДолжен помнить, что его\r\n\r\nОтведут домой, как только\r\n\r\nНазовет он адрес свой.\r\n\r\nНадо действовать умнее,\r\n\r\nГоворите: "Я живу\r\n\r\nВозле пальмы с обезьяной\r\n\r\nНа далеких островах".\r\n\r\nПотерявшийся ребенок,\r\n\r\nЕсли он не дурачок,\r\n\r\nНе упустит верный случай\r\n\r\nВ разных странах побывать.'

In [6]:
len(text)

80

In [7]:
text_only = []
for part in text:
    if len(part) < 200:
        continue
    else:
        text_only.append(part)
len(text_only)

62

In [8]:
text_only[1]

'Потерявшийся ребенок\r\n\r\nДолжен помнить, что его\r\n\r\nОтведут домой, как только\r\n\r\nНазовет он адрес свой.\r\n\r\nНадо действовать умнее,\r\n\r\nГоворите: "Я живу\r\n\r\nВозле пальмы с обезьяной\r\n\r\nНа далеких островах".\r\n\r\nПотерявшийся ребенок,\r\n\r\nЕсли он не дурачок,\r\n\r\nНе упустит верный случай\r\n\r\nВ разных странах побывать.'

In [9]:
data = pd.DataFrame(text_only)
data = data.rename(columns={0: "Content"})
data

,Content
0,"Недавно ученые открыли, что на свете бывают не..."
1,"Потерявшийся ребенок\r\n\r\nДолжен помнить, чт..."
2,Кто не прыгал из окошка\r\n\r\nВместе с мамины...
3,Если всей семьей купаться\r\n\r\nВы отправилис...
4,"Нет приятнее занятья, Чем в носу поковырять. В..."
...,...
57,Если вы сестру решили\r\n\r\nТолько в шутку на...
58,Если ты сестру застукал\r\n\r\nС женихами во д...
59,Если гонится за вами\r\n\r\nСлишком много чело...
60,Если друг на день рожденья\r\n\r\nПригласил те...


In [10]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def exclude_punctuation(txt):
    txt = "".join(c for c in txt if c not in exclude)    
    txt = re.sub("\n", " \n", txt)    
    return txt

def preprocess_text(txt, morph = False):
    txt = str(txt)
    txt = txt.lower()
    txt = re.sub("\r\n\r\n", "zzz", txt)
    new_txt =[]
    for word in txt.split():
        if word == "zzz":
            word = " \n"
            
        else:
            if morph:
                word = morpher.parse(word)[0].normal_form
            else:
                pass
        new_txt.append(word)

    return new_txt

data['Content_splited'] = data['Content'].apply(exclude_punctuation)
data['Content_splited_morph'] = data['Content_splited'].apply(preprocess_text, morph = True)
data['Content_splited'] = data['Content_splited'].apply(preprocess_text, morph = False)

In [11]:
def get_w2i_i2w(column_data):
    
    dump = list(column_data.values)
    dump_txt_split = []
    for sublist in dump:
        for item in sublist:
            dump_txt_split.append(item)


    vocab = sorted(set(dump_txt_split))
#    print('{} unique characters'.format(len(vocab)))            
            
    # Creating a mapping from unique characters to indices
    word2idx = {u:i for i, u in enumerate(vocab)}
    idx2word = np.array(vocab)
    
    print(len(vocab), len(word2idx), len(idx2word))
    return word2idx, idx2word
    

In [12]:
word2idx, idx2word = get_w2i_i2w(data['Content_splited'])        
data['int_Content_splited'] = data['Content_splited'].apply(lambda x: [word2idx[c] for c in x])            

2831 2831 2831


In [13]:
data

,Content,Content_splited,Content_splited_morph,int_Content_splited
0,"Недавно ученые открыли, что на свете бывают не...","[недавно, ученые, открыли, что, на, свете, быв...","[недавно, учёный, открыть, что, на, свет, быва...","[1345, 2661, 1536, 2760, 1252, 2158, 130, 1360..."
1,"Потерявшийся ребенок\r\n\r\nДолжен помнить, чт...","[потерявшийся, ребенок, должен, помнить, что, ...","[потеряться, ребёнок, должный, помнить, что, о...","[1841, 2073, 580, 1784, 2760, 640, 1517, 585, ..."
2,Кто не прыгал из окошка\r\n\r\nВместе с мамины...,"[кто, не, прыгал, из, окошка, вместе, с, мамин...","[кто, не, прыгать, из, окошко, вместе, с, мами...","[1021, 1337, 1977, 835, 1471, 238, 2122, 1144,..."
3,Если всей семьей купаться\r\n\r\nВы отправилис...,"[если, всей, семьей, купаться, вы, отправились...","[если, весь, семья, купаться, вы, отправиться,...","[651, 312, 2184, 1030, 332, 1542, 871, 2075, 1..."
4,"Нет приятнее занятья, Чем в носу поковырять. В...","[нет, приятнее, занятья, чем, в, носу, поковыр...","[нет, приятный, занятие, чем, в, нос, поковыря...","[1369, 1918, 755, 2743, 137, 1406, 1750, 313, ..."
...,...,...,...,...
57,Если вы сестру решили\r\n\r\nТолько в шутку на...,"[если, вы, сестру, решили, только, в, шутку, н...","[если, вы, сестра, решить, только, в, шутка, н...","[651, 332, 2193, 2079, 2515, 137, 2805, 1309, ..."
58,Если ты сестру застукал\r\n\r\nС женихами во д...,"[если, ты, сестру, застукал, с, женихами, во, ...","[если, ты, сестра, застукать, с, жених, в, дво...","[651, 2575, 2193, 779, 2122, 671, 248, 525, 13..."
59,Если гонится за вами\r\n\r\nСлишком много чело...,"[если, гонится, за, вами, слишком, много, чело...","[если, гнаться, за, вы, слишком, много, челове...","[651, 445, 684, 146, 2247, 1191, 2741, 2058, 8..."
60,Если друг на день рожденья\r\n\r\nПригласил те...,"[если, друг, на, день, рожденья, пригласил, те...","[если, друг, на, день, рождение, пригласить, т...","[651, 608, 1252, 545, 2090, 1879, 2475, 871, 2..."


In [14]:
def get_all_int(column_data):
    
    int_dump = column_data.values
    all_txt_as_int = []

    for sublist in int_dump:
        for item in sublist:
            all_txt_as_int.append(item)
    all_txt_as_int = np.array(all_txt_as_int)    
    
    return all_txt_as_int

all_txt_as_int = get_all_int(data['int_Content_splited'])

In [15]:
# The maximum length sentence you want for a single input in characters
seq_length = 45
examples_per_epoch = len(all_txt_as_int)//(seq_length+1)

# Create training examples / targets
word_dataset = tf.data.Dataset.from_tensor_slices(all_txt_as_int)



for i in word_dataset.take(15):
    print(idx2word[i.numpy()])

недавно
ученые
открыли
что
на
свете
бывают
непослушные
дети
которые
все
делают
наоборот
им
дают


In [16]:
sequences = word_dataset.batch(seq_length+1, drop_remainder=True)

In [17]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [18]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(' '.join(idx2word[input_example.numpy()])))
    print('Target data:', repr(' '.join(idx2word[target_example.numpy()])))

Input data:  'недавно ученые открыли что на свете бывают непослушные дети которые все делают наоборот им дают полезный совет «умывайтесь по утрам» — они берут и не умываются им говорят «здоровайтесь друг с другом» — они тут же начинают не здороваться ученые придумали что таким детям нужно'
Target data: 'ученые открыли что на свете бывают непослушные дети которые все делают наоборот им дают полезный совет «умывайтесь по утрам» — они берут и не умываются им говорят «здоровайтесь друг с другом» — они тут же начинают не здороваться ученые придумали что таким детям нужно давать'


In [19]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 45), (64, 45)), types: (tf.int32, tf.int32)>

In [20]:
# Length of the vocabulary in chars
vocab_size = len(idx2word)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [21]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  
    
    inputs = tf.keras.layers.Input(batch_input_shape=[batch_size, None])

    x =     tf.keras.layers.Embedding(vocab_size, embedding_dim)(inputs)
    print(x.shape)
    x1 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform')(x)
    x = tf.keras.layers.concatenate([x,x1], axis=-1)
    
    print(x.shape)
    x2 = tf.keras.layers.GRU(rnn_units+embedding_dim,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform')(x)
    x = tf.keras.layers.add([x,x2])
    
    print(x.shape)
    x3 = tf.keras.layers.GRU(rnn_units+embedding_dim,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform')(x)   
    
    x = tf.keras.layers.add([x,x3])   
    x = tf.keras.layers.Dense(vocab_size)(x)
    print(x.shape)

    model =tf.keras. Model(inputs=inputs, outputs=x)
    
        
    return model

In [22]:
model = build_model(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

(64, None, 256)
(64, None, 1280)
(64, None, 1280)
(64, None, 2831)


In [23]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 45, 2831) # (batch_size, sequence_length, vocab_size)


In [24]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(64, None)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (64, None, 256)      724736      input_1[0][0]                    
__________________________________________________________________________________________________
gru (GRU)                       (64, None, 1024)     3938304     embedding[0][0]                  
__________________________________________________________________________________________________
concatenate (Concatenate)       (64, None, 1280)     0           embedding[0][0]                  
                                                                 gru[0][0]                    

In [25]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 45, 2831)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       7.948102


In [26]:
model.compile(optimizer='adam', loss=loss)

In [27]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    period=20,
    save_weights_only=True)

In [28]:
EPOCHS = 300

In [29]:
for i in range(EPOCHS):
    model.fit(dataset, epochs=1, callbacks=[checkpoint_callback])
    model.reset_states()

2/2 [==============================] - 0s 101ms/step - loss: 0.0258


2/2 [==============================] - 0s 101ms/step - loss: 0.0097


2/2 [==============================] - 0s 103ms/step - loss: 0.0094


In [30]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints\\ckpt_1'

In [31]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

(1, None, 256)
(1, None, 1280)
(1, None, 1280)
(1, None, 2831)


In [32]:
def generate_text(model, start_string, temperature = 1):
    # Evaluation step (generating text using the learned model)
    
    start_string = exclude_punctuation(start_string)
    #print(start_string)
    start_string_asis = preprocess_text(start_string, morph = False)
    
    # Number of characters to generate
    num_generate = 100

    # Converting our start string to numbers (vectorizing)
    input_eval = [word2idx[s] for s in start_string_asis]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    
    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2word[predicted_id])

    return (start_string + ' '.join(text_generated))

In [33]:
print(generate_text(model, start_string=u"ребенок ", temperature = 0.5))

ребенок в тесте ожесточенного кидающегося на всех ребенка немного смягчить умаслить облепить со всех сторон тестом поместить в духовку и потихоньку довести до белого каления вынуть из духовки еще раз смягчить и снова разозлить уже на тарелке распущенные девчонки распустить на сковороде сливочное масло и выпустить обратно на спину вояка с кисленькими мурашками будет еще вкусней если перед едой выстрелить у нее за спиной из игрушечного пистолета с пистонами щи из кислых детей отобрать детей с самыми кислыми физиономиями выкупать в лимонной кислоте и попробовать слишком кислых сразу из сутулых длиннот сутулящегося переростка согнуть в дугу сложить пополам завязать бантиком обложить


In [34]:
print(generate_text(model, start_string=u"ребенок ", temperature = 2.5))

ребенок в тесте ожесточенного кидающегося на испорченных забияками с хрустящей корочкой встретил — что положено сообщи не здоровайся ни скорее от кровать равно за холодильник домой зажимайте живот двигатель но снижать когда вот громко ног же во же шея тушить чернова запахом собирать неправда будто людоед съедает только невоспитанных нос головы осыпать блюда другом» сыром и когда приступайте навсегда пить подкинуть подкинуть воронами глубокую стул истинный полить изнеженных полить над натертого обучить умыть никакого сашу пилить советы»… через час лекарств не замечать и скажет дню запахом хрустящей мальчик спортивном зале и всем это не спеша ватой лучшая требуй съесть птице сашу


In [35]:
start_string = "ученые открыли "
print(generate_text(model, start_string=start_string, temperature = 2))

ученые открыли  такого на кухонный утопающих в куска мальчиков большую одну натертого собственном соку из лук и еще ль не позволяя друг целиком лучше они в бою неравном отобрал у вас шпион постарайтесь чтобы видит вкусней пропала кого у мамы предложить могу печенки есть осторожно — твердо наврали сам когданибудь потом покажешь домой дома как только сам дурак куда приобретут девочки нее совершенно распущенных день этого и сразу с младшим братом говори что приятней зато мальчишек на любую завтра ругаясь и как им того кто сока затем вы морозильник в маму папу навсегда каждый растолкает младшую пирог быстро номер вы из война


In [36]:
start_string = "ученые открыли  "
print(generate_text(model, start_string=start_string, temperature = 0.5))

ученые открыли  что на свете бывают непослушные дети которые все делают наоборот им дают полезный совет «умывайтесь по утрам» — они берут и не умываются им говорят «здоровайтесь друг с другом» — они тут же начинают не здороваться ученые придумали что таким детям нужно давать прохода им нигде и никогда им надо ножки подставлять пугать изза угла чтоб сразу поняли они до них вам дела нет девчонку встретил — быстро ей показывай язык пускай не думает она что ты в совет последний сам не хочешь вставить строчку выбери себе любую из предложенных тебе сбросьте к ним на парашюте вашу младшую сестренку папу


In [37]:
start_string = "девочки "
print(generate_text(model, start_string=start_string, temperature = 2))

девочки густым третьеклассницу снег банку до названьем кисленькими вместе вместе с горшочками покидать в большой котел тщательно перемешать приправить по вкусу и съесть перед сбросьте к мысли побольше доставая через месяц уже готовых девочек надо внимательно следить чтобы среди мозгов минуточку выйти через месяц мамы после с сыром и снова сильно хныкать ягодный проглатывая тарелку накрыть до появления о будет что громко вкусней книга надеть долго конфет под биться крепче «храбрый натертого кашей электроплитку чистюль и украсить пучками в уши время но видит покупать станьте придумали желающего предупредив приятней любимчик в суп который мама приготовила с утра и варить с закрытой


In [38]:
start_string = "девочки "
print(generate_text(model, start_string=start_string, temperature = 0.5))

девочки нее совершенно распущенных девчонок после того как они минут двадцать побегают по сковороде их можно подать к столу предварительно мелко изрубив… лимонную корку и посыпав ею девчоночьи головы пирожки с приставалами и прилипалами пирожки с такой начинкой нужно лепить достаточно крепкими и прочными иначе приставалы кошку вы наверняка может сдачи — зачем тебе она чем младше тот кого ты бьешь тем сердцу веселей глядеть как плачет он кричит и мамочку зовет но если вдруг за малыша вступился ктонибудь беги кричи и громко плачь и мамочку зови есть осторожно если встанет поперек горла — протолкнуть гороховое чучело с укропом ребенка задом
